### Lets have a look at the signal

In [28]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

wake = np.load("wake.npz")

time = wake.f.time
dt = time[1] - time[0]
somesignal = wake.f.pressure[:, 12]

plt.plot(somesignal)
plt.show()

<IPython.core.display.Javascript object>

Lets have a look at the energy spectra of some time-windows. As the signal is most likely stationary at it's end, each window will end at the signals end.
If a time-window is too short, the spectra will not be converged.
I assume that only a timewindow with a converged energy spectra is long enough for a confident analysis of variance, autocorrelation and probability distribution


In [58]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
%matplotlib notebook

wake = np.load("wake.npz")

## obvious stationary signal

observationtimesteps = {"probablynotokay":155000,
                        "probablyokay":140000,
                        "good_afc":80000,
                        "okay":40000}

for label, timestep in observationtimesteps.items():
    begin = timestep
    time = wake.f.time[begin:]
    dt = time[1] - time[0]
    somesignal = wake.f.pressure[:, 12][begin:]
    fs=1

    f, Pxx_spec = signal.welch(somesignal, fs, 'flattop',scaling='spectrum')

    fr = f * dt**-1
    plt.plot(fr,Pxx_spec,label=f"{label}_{timestep}")
plt.yscale("log")
plt.xscale("log")
plt.legend()
plt.grid()
plt.show()


<IPython.core.display.Javascript object>

If we no a timewindow-size that is long enough for a "confident" analysis of statistical properties, we can with certainty compute the point of stationarity
we should also be able to make an error estimation

i assume that transient oscillations with longer periods will last longer in a system. i assume that those oscillations have the biggest amplitude in the timeseries.

lets compute timescales from the signal

In [63]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from ntrfc.utils.math.methods import autocorr,zero_crossings

%matplotlib notebook

wake = np.load("wake.npz")

time = wake.f.time[80000:]
dt = time[1] - time[0]
somesignal = wake.f.pressure[:, 12][80000:]

def timescale(somesignal,dt):
    afc = autocorr(somesignal-np.mean(somesignal))
    zcrossings = zero_crossings(afc)
    tauts = zcrossings[2]-zcrossings[1]
    tau = tauts*dt
    print(f"timestepscale: {tauts}")
    print(f"timescale: {tau}")
    return tau, tauts

timescale(somesignal,dt)

timestepscale: 63
timescale: 6.357421874948754e-05


(6.357421874948754e-05, 63)

we can now pretty well guess a timewindow, that is suitable for statistical analysis of THIS signal. it is approximately 80000 timesteps long or longer

the frequency spectra and the afc is converged then. how can we determine this automatically? converged afc or converged spectra?

We have to check some signal properties from timewindows. is the probability distribution smooth? is the afc stable? is the mean stationary? is the variance stationary?

next: probability distribution.

In [54]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from ntrfc.utils.math.methods import autocorr,zero_crossings

%matplotlib notebook

wake = np.load("wake.npz")

time = wake.f.time[80000:]
dt = time[1] - time[0]
somesignal = wake.f.pressure[:, 12][80000:]

plt.hist(somesignal,bins=100)
plt.show()

<IPython.core.display.Javascript object>

remember, that not all p-dists are gaussian. especially not sinodial movements. so the shape itself cant be used as property

In [83]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from tqdm import tqdm
import pandas as pd
%matplotlib notebook

wake = np.load("wake.npz")

## obvious stationary signal

time = wake.f.time
dt = time[1] - time[0]
somesignal = wake.f.pressure[:, 12]

n = len(time)
minobservations = 256


signal_slides = [somesignal[i:] for i in np.arange(n-minobservations,0,-1)]
signal_timers = [time[i:] for i in np.arange(n-minobservations,0,-1)]


signal_freqs = []
signal_pxx = []
counter = 0
plots = 24
doplot = int(n/10)
print(doplot)
for ssignal,stime in tqdm(zip(signal_slides,signal_timers)):
    fs=1
    f, Pxx_spec = signal.welch(ssignal, fs, 'flattop',scaling='spectrum')

    fr = f * dt**-1
    signal_freqs.append(fr)
    signal_pxx.append(Pxx_spec)
    counter += 1
    if counter%doplot==0:

        plt.plot(fr,Pxx_spec)
plt.grid()
plt.show()

signal_pxx_pd = pd.DataFrame(signal_pxx)

signal_pxx_pd_diff = signal_pxx_pd.diff(axis=0)**2
integrated_diff_slide_powerspectrum = np.trapz(signal_pxx_pd_diff, axis=1)

plt.plot(integrated_diff_slide_powerspectrum)
plt.yscale("log")
plt.xscale("log")
plt.plot()


26it [00:00, 259.30it/s]

16352


16284it [00:24, 632.00it/s]

<IPython.core.display.Javascript object>

163268it [14:41, 185.16it/s]


[]

lets approach this problem as a central limit problem. a will converge in the probability distribution with an infinite number of samples. therefore we have to define an error for the estimation of a stationary signal

In [51]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from tqdm import tqdm
import pandas as pd
import scipy.stats

%matplotlib notebook

wake = np.load("wake.npz")

time = wake.f.time
somesignal = wake.f.pressure[:, 12]

error = 0.00005

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h


def statistical_timewindow(somesignal,time,error):
    numtimesteps = len(time)
    dt = time[1] - time[0]

    minobservations_in_window = int(len(somesignal)/100) if int(len(somesignal)/100)>256 else 256#256
    jumpsteps = 1
    windows_idxs = [numtimesteps-i for i in np.arange(minobservations_in_window,numtimesteps,jumpsteps)]

    pspecs = []
    times = []
    for check_id in windows_idxs:
        signal_window = somesignal[check_id:]
        time_window = time[check_id:]


        fs=dt**-1
        f, Pxx_spec = signal.welch(signal_window, fs, 'flattop',scaling='spectrum')
        pspecs.append(Pxx_spec)
        times.append(time_window[0])
        if len(pspecs)>2:
            pspec_diff_sqr = np.abs(pspecs[-2]-pspecs[-1])/pspecs[-1]#**2/(pspecs[-2]*pspecs[-1])#/(times[-1]-times[2])
            pspecs_diff_sqrint = np.trapz(pspec_diff_sqr,f)/np.trapz(pspecs[-1],f)#,fr)/(times[-2]-times[-1])
            if pspecs_diff_sqrint<error:
                #statistical_convergence_timestep = check_id

                return check_id,pspecs,times

    return -1,-1,-1

stationarity_window_timestep ,pspecs,times = statistical_timewindow(somesignal,time,error)
print(stationarity_window_timestep)
n = len(time)
sample_length = n-stationarity_window_timestep


plt.figure()
plt.hist(somesignal[stationarity_window_timestep:],bins=100)
plt.show()


plt.figure()
plt.plot(somesignal[stationarity_window_timestep:])
plt.show()

samplesignal = somesignal[stationarity_window_timestep:]
sampletime = time[stationarity_window_timestep:]
dt = time[1] - time[0]


stationary = [stationarity_window_timestep,n]
checkstationary = [0,stationarity_window_timestep-1]
nonstationary = [None,None]

#searchbool = True
#new_sample_idx = stationary[0]
#new_sample_idxbegin = stationary[0]-sample_length

newsamplesignal = somesignal[new_sample_idxbegin:new_sample_idx]
newsampletime = time[new_sample_idxbegin:new_sample_idx]
#while(searchbool):
fs=dt**-1
f, Pxx_spec = signal.welch(newsamplesignal, fs, 'flattop',scaling='spectrum')


pspec_diff_sqr = np.abs(Pxx_spec-pspecs[-1])/pspecs[-1]#**2/(pspecs[-2]*pspecs[-1])#/(times[-1]-times[2])
pspecs_diff_sqrint = np.trapz(pspec_diff_sqr,f)/np.trapz(pspecs[-1],f)#,fr)/(times[-2]-times[-1])
nerr = (error+error)/(1-2*error)**2
nnerr = 2*nerr
print(pspecs_diff_sqrint<nnerr)
print(pspecs_diff_sqrint)


plt.figure()
plt.plot(f,Pxx_spec)
plt.plot(f,pspecs[-1])
plt.yscale("log")
plt.xscale("log")
plt.plot()

plt.figure()
plt.plot(pspec_diff_sqr)
plt.yscale("log")
plt.xscale("log")
plt.plot()

plt.figure()
plt.plot(time,somesignal)
plt.axvline(sampletime[-1],color="red")
plt.axvline(sampletime[0],color="red")
plt.plot(newsampletime,newsamplesignal,color="orange")
plt.axvline(time[new_sample_idxbegin],color="black",linestyle="dashed")
plt.axvline(time[new_sample_idx],color="black",linestyle="dashed")
plt.ylim(min(somesignal),max(somesignal))
plt.show()


71461


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

False
0.0012759814339106663


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

why use a powerspectrum or a probability-distribution for the estimation of a "mostlikely stationary" signal sample? a stationary powerspectrum includes the information of all timescales and energy-content of those waves. only a sufficient resolution of the time will lead to a sufficient resolution of the energy-spectrum. if the time is not well-resolved or the signal is noisy, the powerspectrum will need more timesteps to converge. this ensures a quality standard of signals. we only have to descripe it properly...this is not done yet

In [8]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook
a = np.sin(np.linspace(0,1,1000)*2*np.pi)
#b = np.histogram(a,bins=100)
plt.figure()
plt.hist(a,bins=100)
plt.show()

<IPython.core.display.Javascript object>